In [3]:
import pandas as pd

books = pd.read_csv("books_with_categories.csv")

In [2]:
from transformers import pipeline
classifier = pipeline("text-classification",
                    model="j-hartmann/emotion-english-distilroberta-base",
                    top_k = None,
                    device = 0)
classifier("I love this!")

Device set to use cuda:0


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528684265911579},
  {'label': 'neutral', 'score': 0.0057645998895168304},
  {'label': 'anger', 'score': 0.004419790115207434},
  {'label': 'sadness', 'score': 0.002092393347993493},
  {'label': 'disgust', 'score': 0.0016119939973577857},
  {'label': 'fear', 'score': 0.0004138524236623198}]]

In [4]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [5]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548421382904053},
  {'label': 'neutral', 'score': 0.1698516607284546},
  {'label': 'sadness', 'score': 0.11640840768814087},
  {'label': 'surprise', 'score': 0.020700694993138313},
  {'label': 'disgust', 'score': 0.019100673496723175},
  {'label': 'joy', 'score': 0.015161201357841492},
  {'label': 'anger', 'score': 0.003935144282877445}]]

In [6]:
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.729603111743927},
  {'label': 'neutral', 'score': 0.14038562774658203},
  {'label': 'fear', 'score': 0.06816202402114868},
  {'label': 'joy', 'score': 0.04794226214289665},
  {'label': 'anger', 'score': 0.009156344458460808},
  {'label': 'disgust', 'score': 0.002628471702337265},
  {'label': 'sadness', 'score': 0.002122156322002411}],
 [{'label': 'neutral', 'score': 0.44937142729759216},
  {'label': 'disgust', 'score': 0.2735912501811981},
  {'label': 'joy', 'score': 0.10908282548189163},
  {'label': 'sadness', 'score': 0.093626968562603},
  {'label': 'anger', 'score': 0.04047830402851105},
  {'label': 'surprise', 'score': 0.026970187202095985},
  {'label': 'fear', 'score': 0.00687905540689826}],
 [{'label': 'neutral', 'score': 0.6462156176567078},
  {'label': 'sadness', 'score': 0.24273362755775452},
  {'label': 'disgust', 'score': 0.04342270642518997},
  {'label': 'surprise', 'score': 0.02830052562057972},
  {'label': 'joy', 'score': 0.0142114562913

In [12]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [13]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [14]:
predictions[0]

[{'label': 'surprise', 'score': 0.729603111743927},
 {'label': 'neutral', 'score': 0.14038562774658203},
 {'label': 'fear', 'score': 0.06816202402114868},
 {'label': 'joy', 'score': 0.04794226214289665},
 {'label': 'anger', 'score': 0.009156344458460808},
 {'label': 'disgust', 'score': 0.002628471702337265},
 {'label': 'sadness', 'score': 0.002122156322002411}]

In [15]:
sentences[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [16]:
predictions[3]

[{'label': 'fear', 'score': 0.928168535232544},
 {'label': 'anger', 'score': 0.03219069913029671},
 {'label': 'neutral', 'score': 0.012808645144104958},
 {'label': 'sadness', 'score': 0.008756839670240879},
 {'label': 'surprise', 'score': 0.008597890846431255},
 {'label': 'disgust', 'score': 0.008431789465248585},
 {'label': 'joy', 'score': 0.0010455796727910638}]

In [17]:
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"]
isbn = []
emotion_scores = {label:[] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label:[] for label in emotion_labels}
    for prediction in predictions:
        sorted_prediction = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_prediction[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [18]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_score = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_score[label])


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [19]:
emotion_scores

{'anger': [np.float64(0.06413362175226212),
  np.float64(0.612618088722229),
  np.float64(0.06413362175226212),
  np.float64(0.351483553647995),
  np.float64(0.08141222596168518),
  np.float64(0.23222492635250092),
  np.float64(0.5381852984428406),
  np.float64(0.06413362175226212),
  np.float64(0.3006702959537506),
  np.float64(0.06413362175226212)],
 'disgust': [np.float64(0.2735912501811981),
  np.float64(0.34828561544418335),
  np.float64(0.10400664806365967),
  np.float64(0.15072225034236908),
  np.float64(0.18449515104293823),
  np.float64(0.7271747589111328),
  np.float64(0.1558549553155899),
  np.float64(0.10400664806365967),
  np.float64(0.27948132157325745),
  np.float64(0.1779283583164215)],
 'fear': [np.float64(0.928168535232544),
  np.float64(0.9425276517868042),
  np.float64(0.9723208546638489),
  np.float64(0.3607068657875061),
  np.float64(0.09504342079162598),
  np.float64(0.05136280506849289),
  np.float64(0.7474280595779419),
  np.float64(0.40449678897857666),
  np.f

In [20]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"]
isbn = []
emotion_scores = {label:[] for label in emotion_labels}
for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_score = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_score[label])

100%|██████████| 5197/5197 [05:41<00:00, 15.21it/s]


In [21]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [22]:
emotions_df

,anger,disgust,fear,joy,neutral,sadness,surprise,isbn13
0,0.064134,0.273591,0.928169,0.932798,0.646216,0.967158,0.729603,9780002005883
1,0.612618,0.348286,0.942528,0.704422,0.887939,0.111690,0.252544,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765,9780006178736
3,0.351484,0.150722,0.360707,0.251881,0.732686,0.111690,0.078765,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.148208,0.030643,0.919165,0.255170,0.853721,0.980877,0.030656,9788172235222
5193,0.064134,0.114383,0.051363,0.400262,0.883199,0.111690,0.227765,9788173031014
5194,0.009997,0.009929,0.339217,0.947779,0.375756,0.066685,0.057625,9788179921623
5195,0.064134,0.104007,0.459269,0.759456,0.951104,0.368111,0.078765,9788185300535


In [23]:
books = pd.merge(books, emotions_df, on="isbn13")

In [24]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,neutral,sadness,surprise
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273591,0.928169,0.932798,0.646216,0.967158,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612618,0.348286,0.942528,0.704422,0.887939,0.111690,0.252544
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351484,0.150722,0.360707,0.251881,0.732686,0.111690,0.078765
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.148208,0.030643,0.919165,0.255170,0.853721,0.980877,0.030656
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.064134,0.114383,0.051363,0.400262,0.883199,0.111690,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.009997,0.009929,0.339217,0.947779,0.375756,0.066685,0.057625
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.064134,0.104007,0.459269,0.759456,0.951104,0.368111,0.078765


In [25]:
books.to_csv("books_with_emotions.csv", index=False)